# Scraping Example using Scrapy

Basic example for scraping quotes from quotes.toscrape.com. Example code from https://www.jitsejan.com/using-scrapy-in-jupyter-notebook.html

## Import Scrapy

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

## Setup a pipeline

This class creates a simple pipeline that writes all found items to a JSON file, where each line contains one JSON element.

In [2]:
import os
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        if not os.path.exists('./data'):
            os.mkdir('./data')
        self.file = open('./data/quoteresult.jsonl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

## Define the spider

The QuotesSpider class defines from which URLs to start crawling and which values to retrieve. I set the logging level of the crawler to warning, otherwise the notebook is overloaded with DEBUG messages about the retrieved data.

In [3]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': './data/quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

## Start the crawler

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2020-09-29 10:43:07 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-09-29 10:43:07 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.4 (default, Oct 24 2019, 11:00:43) - [Clang 11.0.0 (clang-1100.0.33.8)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Darwin-18.5.0-x86_64-i386-64bit
2020-09-29 10:43:07 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-09-29 10:43:07 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2020-09-29 10:43:07 [py.warnings] WARNING: /Users/alejo/Code/MECEE4520_2020/env/lib/python3.7/site-packages/scrapy/extensions/feedexport.py:239: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
 

## Check the files

Verify that the files has been created on disk. As we can observe the files are both created and have data. The .jl file has line separated JSON elements, while the .json file has one big JSON array containing all the quotes.

In [5]:
ll ./data/quoteresult.*

-rw-r--r--  1 alejo  staff  11146 Sep 29 10:43 ./data/quoteresult.json
-rw-r--r--  1 alejo  staff   5551 Sep 29 10:43 ./data/quoteresult.jsonl


In [6]:
!tail -n 2 ./data/quoteresult.jsonl

{"text": "\u201cGood friends, good books, and a sleepy conscience: this is the ideal life.\u201d", "author": "Mark Twain", "tags": ["books", "contentment", "friends", "friendship", "life"]}
{"text": "\u201cLife is what happens to us while we are making other plans.\u201d", "author": "Allen Saunders", "tags": ["fate", "life", "misattributed-john-lennon", "planning", "plans"]}


In [7]:
!tail -n 2 ./data/quoteresult.json

{"text": "\u201cLife is what happens to us while we are making other plans.\u201d", "author": "Allen Saunders", "tags": ["fate", "life", "misattributed-john-lennon", "planning", "plans"]}
]